In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5931, done.
remote: Total 5931 (delta 0), reused 0 (delta 0), pack-reused 5931
Receiving objects: 100% (5931/5931), 6.34 MiB | 24.15 MiB/s, done.
Resolving deltas: 100% (3925/3925), done.


In [3]:
!git clone https://github.com/sleepless-se/keras-yolo3.git keras-yolo3-sample

Cloning into 'keras-yolo3-sample'...
remote: Enumerating objects: 211, done.
remote: Total 211 (delta 0), reused 0 (delta 0), pack-reused 211
Receiving objects: 100% (211/211), 161.77 KiB | 4.15 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [4]:
cd keras-yolo3-sample

/content/keras-yolo3-sample


In [5]:
!pip install -r requirements.txt

     |████████████████████████████████| 102kB 4.2MB/s 
     |████████████████████████████████| 2.2MB 22.2MB/s 
     |████████████████████████████████| 2.8MB 41.1MB/s 
     |████████████████████████████████| 890kB 53.4MB/s 
     |████████████████████████████████| 337kB 43.1MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 92kB 12.1MB/s 
     |████████████████████████████████| 13.0MB 253kB/s 
     |████████████████████████████████| 17.3MB 214kB/s 
     |████████████████████████████████| 2.0MB 47.8MB/s 
     |████████████████████████████████| 1.2MB 48.2MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
     |████████████████████████████████| 235kB 52.6MB/s 
     |████████████████████████████████| 276kB 29.1MB/s 
     |████████████████████████████████| 25.2MB 138kB/s 
     |████████████████████████████████| 3.1MB 47.0MB/s 
     |████████████████████████████████| 45.9MB 68kB/s 
     |████████████████████████████████| 327kB 47.8MB/

In [8]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [7]:
cd /content/keras-yolo3-sample/

/content/keras-yolo3-sample


In [16]:
import os
import pandas
main_dir = 'VOCDevkit/VOC2007/ImageSets/Main/'
os.chdir(main_dir)
pwd = os.getcwd()
print(pwd)


def make_train_files():
    suffixs = ['_train','_val','_test']

    for suffix in suffixs:
        print('suffix',suffix)
        new_file = open('{}.txt'.format(suffix.replace('_','')),'w')
        text = ""
        for file in os.listdir():
            if file.find(suffix) == -1:continue
            with open(file) as f:
                if text == "":text = f.read()
                text =text +'\n'+ f.read()
        new_file.write(text)

def split_val_test(rate:float):
    val = pandas.read_csv('val.txt')
    val = val.sample(frac=1)
    print('len',len(val))
    split = int(len(val) * rate)
    test = val.values
    val[:split].to_csv('val.txt',index=False)
    val[split:].to_csv('test.txt',index=False)
    print('val.txt',split)
    print('test.txt',len(val) - split)

if __name__ == '__main__':
    val_rate = 0.33
    make_train_files()
    split_val_test(val_rate)

/content/keras-yolo3-sample/VOCDevkit/VOC2007/ImageSets/Main
suffix _train
suffix _val
suffix _test
len 19
val.txt 6
test.txt 13


In [21]:
cd ..

/content/keras-yolo3-sample


In [22]:
import xml.etree.ElementTree as ET
from os import getcwd
import sys,os
sets=[('2007', 'train'), ('2007', 'val'), ('2007', 'test')]

classes = ["solar_panel"]

if len(sys.argv) > 1:
    classes = sys.argv[1:]

with open('model_data/voc_classes.txt','w') as f:
    f.write('\n'.join(classes))


def convert_annotation(year, image_id, list_file):
    in_file = open('VOCDevkit/VOC%s/Annotations/%s.xml'%(year, image_id.replace(".jpg","")))
    tree=ET.parse(in_file)
    root = tree.getroot()

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (int(float(xmlbox.find('xmin').text)),
             int(float(xmlbox.find('ymin').text)),
             int(float(xmlbox.find('xmax').text)),
             int(float(xmlbox.find('ymax').text)))
        list_file.write(" " + ",".join([str(a) for a in b]) + ',' + str(cls_id))

wd = getcwd()

for year, image_set in sets:
    image_ids = open('VOCDevkit/VOC%s/ImageSets/Main/%s.txt'%(year, image_set)).read().strip().split()
    list_file = open('model_data/%s_%s.txt'%(year, image_set), 'w')
    for image_id in image_ids:
        if image_id == '1': continue
        if image_id == '-1': continue
        image_file_path = '%s/VOCDevkit/VOC%s/JPEGImages/%s'%(wd, year, image_id)
        list_file.write(image_file_path)
        convert_annotation(year, image_id, list_file)
        list_file.write('\n')
    list_file.close()

In [47]:
print(os.getcwd())

/content/keras-yolo3


In [56]:
!pip install keras

In [13]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-02-05 08:23:36--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   377KB/s    in 9m 3s   

2021-02-05 08:32:39 (446 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [14]:
!python convert.py -w yolov3.cfg yolov3.weights model_data/yolo_weights.h5


Using TensorFlow backend.
Loading weights.
Weights Header:  0 2 0 [32013312]
Parsing Darknet config.
Creating Keras model.
Parsing section net_0
Parsing section convolutional_0
conv2d bn leaky (3, 3, 3, 32)
2021-02-05 08:32:45.177463: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Parsing section convolutional_1
conv2d bn leaky (3, 3, 32, 64)
Parsing section convolutional_2
conv2d bn leaky (1, 1, 64, 32)
Parsing section convolutional_3
conv2d bn leaky (3, 3, 32, 64)
Parsing section shortcut_0
Parsing section convolutional_4
conv2d bn leaky (3, 3, 64, 128)
Parsing section convolutional_5
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_6
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_1
Parsing section convolutional_7
conv2d bn leaky (1, 1, 128, 64)
Parsing section convolutional_8
conv2d bn leaky (3, 3, 64, 128)
Parsing section shortcut_2
Parsing section convolutional

In [1]:
!cat /proc/uptime | awk '{print $1 /60 /60 "hours"}'

0.0882556hours


In [8]:
cd ..

/


!python train.py

In [20]:
!python train.py

Using TensorFlow backend.
2021-02-04 07:08:59.024867: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Create YOLOv3 model with 9 anchors and 2 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of

In [24]:
!python train.py

Using TensorFlow backend.
2021-02-04 09:53:29.006461: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Create YOLOv3 model with 9 anchors and 2 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of

In [23]:
!nvidia-smi

Thu Feb  4 09:47:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python train.py


Using TensorFlow backend.
2021-02-04 11:59:24.598313: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Create YOLOv3 model with 9 anchors and 2 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [20]:
!python train.py


Using TensorFlow backend.
2021-02-05 00:42:47.445873: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Create YOLOv3 model with 9 anchors and 1 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 18) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((18,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 18) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of

In [ ]:
!python train.py

Using TensorFlow backend.
2021-02-05 08:37:03.887574: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Create YOLOv3 model with 9 anchors and 2 classes.
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 21) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((21,) vs (255,)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 21) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:3462: UserWarning: Skipping loading of